In [1]:
import folium
import pandas as pd 
import numpy as np

from bokeh.palettes import plasma 

import branca
import branca.colormap as cm

In [8]:
df = pd.read_csv("../Data/AB_data_clean.csv").dropna(subset=['latitude', 'longitude'])
df_trees = pd.read_csv("../Data/trees_data_clean.csv").dropna(subset=['latitude', 'longitude'])
df_rats = pd.read_csv("../Data/rats_data_clean.csv").dropna(subset=['latitude', 'longitude'])
df_places = pd.read_csv("../Data/places_data_clean.csv").dropna(subset=['latitude', 'longitude'])

In [9]:
print('Datapoints in Air BnB data:', df.shape[0])
print('Datapoints in Tree data:', df_trees.shape[0])
print('Datapoints in Rat data:', df_rats.shape[0])
print('Datapoints in Touristic data:', df_places.shape[0])

Datapoints in Air BnB data: 48713
Datapoints in Tree data: 213043
Datapoints in Rat data: 14307
Datapoints in Touristic data: 314


In [41]:
# Making sure the relative frequency of the three data types are preserved in the plot
data_price = df.sample(round(df.shape[0]/100), random_state=3)
data_tree = df_trees.sample(round(df_trees.shape[0]/100), random_state=3)
data_rats = df_rats.sample(round(df_rats.shape[0]/10), random_state=3)
data_places = df_places.sample(round(df_places.shape[0]/1), random_state=3)

print('Datapoints in Air BnB plot data:', data_price.shape[0])
print('Datapoints in Tree plot data:', data_tree.shape[0])
print('Datapoints in Rat plot data:', data_rats.shape[0])
print('Datapoints in Touristic Places plot data:', data_places.shape[0])

Datapoints in Air BnB plot data: 487
Datapoints in Tree plot data: 2130
Datapoints in Rat plot data: 1431
Datapoints in Touristic Places plot data: 314


## Version 2

In [52]:
p_min = data_price.price.min(); p_max = data_price.price.max()
colormap = cm.LinearColormap(colors=['red','pink','lightblue','blue'], 
                             index=[p_min,100,200,p_max],
                             vmin=p_min,vmax=p_max,
                             caption='Price'
                            )

In [53]:
#creating to different marker groups, one for each time period
BnB_prices = folium.FeatureGroup(name='<span style=\\"color: #0C0786;\\">Air BnBs </span>')
Trees = folium.FeatureGroup(name='<span style=\\"color: #79D151;\\">Trees</span>', show = False)
Rats = folium.FeatureGroup(name='<span style=\\"color: #C0C0C0;\\">Rat Sightings</span>', show = False)
Places = folium.FeatureGroup(name='<span style=\\"color: #5AD8CD;\\">Touristic Places</span>', show = False)

In [54]:
def add_marker_price(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 2, color=colormap(x.price), weight = 2,  fill_color=colormap(x.price), fill=True, fill_opacity=1).add_to(BnB_prices)

def add_marker_tree(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#79D151', weight = 2,  fill_color='#79D151', fill=True, fill_opacity=1).add_to(Trees)

def add_marker_rat(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#333436', weight = 2,  fill_color='#333436', fill=True, fill_opacity=1).add_to(Rats)

def add_marker_place(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#f29d0a', weight = 2,  fill_color='#f29d0a', fill=True, fill_opacity=1).add_to(Places)

In [55]:
NYC_coor = (40.730610, -73.935242)

NY_map = folium.Map(NYC_coor, zoom_start=11, tiles = 'cartodbpositron')
folium.TileLayer('cartodbpositron').add_to(NY_map)

#apply the function we just created to every row in the dataframe
data_tree.apply(lambda x:  add_marker_tree(x), axis =1)
data_rats.apply(lambda x:  add_marker_rat(x), axis =1)
data_price.apply(lambda x:  add_marker_price(x), axis =1)
data_places.apply(lambda x:  add_marker_place(x), axis =1)

#add borh marker groups to the map
BnB_prices.add_to(NY_map)
Places.add_to(NY_map)
Rats.add_to(NY_map)
Trees.add_to(NY_map)


NY_map.add_child(colormap)
folium.map.LayerControl('topright', collapsed=False).add_to(NY_map)
NY_map.save('Map_features.html')
#display map
NY_map